In [1]:
#Imports
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
# import hvplot.pandas

# from path import Path
# import plotly.express as px

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


# Postgres driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
url = 'https://raw.githubusercontent.com/joshuanallen/Air_Quality_Prediction/daaronson/air_data.csv'
df = pd.read_csv(url)
# Dataset is now stored in a Pandas Dataframe
# Code found at https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92

Direct read in from CSV below (Legacy)

In [4]:
# # Loading data
# file_path = "air_data.csv"
# df = pd.read_csv(file_path)
# df.head(10)

In [5]:
df.columns.drop("CES 3.0 Score")
df.head()

,Census Tract,Total Population,California County,ZIP,Nearby City \n(to help approximate location only),Longitude,Latitude,CES 3.0 Score,CES 3.0 Percentile,CES 3.0 \nPercentile Range,SB 535 Disadvantaged Community,Ozone,Ozone Pctl,PM2.5,PM2.5 Pctl,Diesel PM,Diesel PM Pctl,Drinking Water,Drinking Water Pctl,Pesticides,Pesticides Pctl,Tox. Release,Tox. Release Pctl,Traffic,Traffic Pctl,Cleanup Sites,Cleanup Sites Pctl,Groundwater Threats,Groundwater Threats Pctl,Haz. Waste,Haz. Waste Pctl,Imp. Water Bodies,Imp. Water Bodies Pctl,Solid Waste,Solid Waste Pctl,Pollution Burden,Pollution Burden Score,Pollution Burden Pctl,Asthma,Asthma Pctl,Low Birth Weight,Low Birth Weight Pctl,Cardiovascular Disease,Cardiovascular Disease Pctl,Education,Education Pctl,Linguistic Isolation,Linguistic Isolation Pctl,Poverty,Poverty Pctl,Unemployment,Unemployment Pctl,Housing Burden,Housing Burden Pctl,Pop. Char.,Pop. Char. Score,Pop. Char. Pctl
0,6019001100,3174,Fresno,93706,Fresno,-119.781696,36.709695,94.09,100.00,95-100% (highest scores),Yes,0.065,98.18,15.40,97.22,48.524,95.54,681.20,80.92,2.75,47.82,18551.957190,97.46,909.14,62.98,80.5,98.67,45.75,89.85,0.795,84.32,0,0.00,21.75,97.81,79.96,9.85,99.95,131.64,97.67,7.44,93.84,14.13,96.31,53.3,95.76,16.2,77.51,76.3,97.12,17.6,91.72,26.0,79.40,92.12,9.55,99.70
1,6071001600,6133,San Bernardino,91761,Ontario,-117.618013,34.057780,90.68,99.99,95-100% (highest scores),Yes,0.062,91.10,13.31,93.64,38.556,92.12,904.66,96.11,1.37,41.34,7494.236622,89.05,782.26,55.66,66.2,97.68,36.00,85.57,1.250,88.77,5,55.01,12.00,92.17,81.19,10.00,100.00,60.66,69.78,7.04,90.85,12.94,92.66,53.3,95.76,33.4,96.25,72.5,94.63,12.3,71.82,34.1,93.75,87.44,9.07,98.11
2,6019000200,3167,Fresno,93706,Fresno,-119.805504,36.735491,85.97,99.97,95-100% (highest scores),Yes,0.062,91.10,15.40,97.22,47.445,95.42,681.20,80.92,3.03,48.75,12454.948410,95.42,576.52,39.00,22.0,85.13,30.25,81.93,0.200,60.50,0,0.00,2.50,57.18,71.16,8.76,99.00,142.12,98.33,10.16,99.78,14.96,97.67,42.3,89.06,16.7,78.39,86.8,99.56,16.1,87.98,40.1,97.85,94.58,9.81,99.99
3,6077000801,6692,San Joaquin,95203,Stockton,-121.314524,37.940517,82.49,99.96,95-100% (highest scores),Yes,0.046,53.02,12.54,84.02,24.117,73.52,278.76,29.11,12.93,60.56,2387.782922,69.97,1305.01,78.29,50.1,96.10,132.10,98.41,0.795,84.32,19,98.63,27.00,99.10,74.48,9.17,99.59,142.17,98.34,6.23,80.65,14.72,97.17,40.8,87.52,15.3,75.14,61.3,85.57,19.6,94.97,21.1,63.54,86.70,8.99,97.72
4,6019001500,2206,Fresno,93725,Fresno,-119.717843,36.681600,82.03,99.95,95-100% (highest scores),Yes,0.065,98.18,15.40,97.22,18.846,58.22,1000.24,98.64,3518.41,95.15,21790.706720,98.15,435.16,24.30,60.0,97.15,54.20,92.09,13.100,99.70,0,0.00,50.80,99.91,80.20,9.88,99.99,90.48,89.54,4.50,38.92,12.82,92.36,45.1,91.13,14.7,73.72,66.4,90.23,18.6,93.65,28.1,83.98,80.08,8.30,92.76


In [6]:
new_df = df[['Census Tract', 'Total Population', 'ZIP', 'Longitude', 'Latitude', 'Ozone', 'PM2.5', 'Diesel PM', 'Pesticides', 'Tox. Release', 'Traffic', 'Asthma', 'Low Birth Weight', 'Cardiovascular Disease', 'Poverty', 'Unemployment']].copy()
new_df.head()

,Census Tract,Total Population,ZIP,Longitude,Latitude,Ozone,PM2.5,Diesel PM,Pesticides,Tox. Release,Traffic,Asthma,Low Birth Weight,Cardiovascular Disease,Poverty,Unemployment
0,6019001100,3174,93706,-119.781696,36.709695,0.065,15.40,48.524,2.75,18551.957190,909.14,131.64,7.44,14.13,76.3,17.6
1,6071001600,6133,91761,-117.618013,34.057780,0.062,13.31,38.556,1.37,7494.236622,782.26,60.66,7.04,12.94,72.5,12.3
2,6019000200,3167,93706,-119.805504,36.735491,0.062,15.40,47.445,3.03,12454.948410,576.52,142.12,10.16,14.96,86.8,16.1
3,6077000801,6692,95203,-121.314524,37.940517,0.046,12.54,24.117,12.93,2387.782922,1305.01,142.17,6.23,14.72,61.3,19.6
4,6019001500,2206,93725,-119.717843,36.681600,0.065,15.40,18.846,3518.41,21790.706720,435.16,90.48,4.50,12.82,66.4,18.6


In [7]:
new_df.set_index('Census Tract')

,Total Population,ZIP,Longitude,Latitude,Ozone,PM2.5,Diesel PM,Pesticides,Tox. Release,Traffic,Asthma,Low Birth Weight,Cardiovascular Disease,Poverty,Unemployment
Census Tract,,,,,,,,,,,,,,,
6019001100,3174,93706,-119.781696,36.709695,0.065,15.400000,48.524,2.75,18551.957190,909.14,131.64,7.44,14.13,76.3,17.6
6071001600,6133,91761,-117.618013,34.057780,0.062,13.310000,38.556,1.37,7494.236622,782.26,60.66,7.04,12.94,72.5,12.3
6019000200,3167,93706,-119.805504,36.735491,0.062,15.400000,47.445,3.03,12454.948410,576.52,142.12,10.16,14.96,86.8,16.1
6077000801,6692,95203,-121.314524,37.940517,0.046,12.540000,24.117,12.93,2387.782922,1305.01,142.17,6.23,14.72,61.3,19.6
6019001500,2206,93725,-119.717843,36.681600,0.065,15.400000,18.846,3518.41,21790.706720,435.16,90.48,4.50,12.82,66.4,18.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6009000504,942,95223,-120.211151,38.405130,0.055,4.645934,0.334,0.76,0.179624,99.33,40.43,NaN,13.11,24.7,NaN
6065940100,166,92239,-114.475335,34.000183,0.044,9.945784,0.226,5.94,2.150919,72.49,88.35,NaN,11.35,21.0,NaN
6053011502,1710,93923,-121.735102,36.301079,0.035,3.991772,0.082,0.00,0.000990,216.01,11.80,NaN,3.79,29.0,NaN


In [8]:
new_df.shape

(8035, 16)

In [9]:
new_df.dtypes

Census Tract                int64
Total Population            int64
ZIP                         int64
Longitude                 float64
Latitude                  float64
Ozone                     float64
PM2.5                     float64
Diesel PM                 float64
Pesticides                float64
Tox. Release              float64
Traffic                   float64
Asthma                    float64
Low Birth Weight          float64
Cardiovascular Disease    float64
Poverty                   float64
Unemployment              float64
dtype: object

In [10]:
#find null values
for column in new_df.columns:
    print(f"Column {column} has {new_df[column].isnull().sum()} null values")

Column Census Tract has 0 null values
Column Total Population has 0 null values
Column ZIP has 0 null values
Column Longitude has 0 null values
Column Latitude has 0 null values
Column Ozone has 0 null values
Column PM2.5 has 19 null values
Column Diesel PM has 0 null values
Column Pesticides has 0 null values
Column Tox. Release has 0 null values
Column Traffic has 56 null values
Column Asthma has 0 null values
Column Low Birth Weight has 222 null values
Column Cardiovascular Disease has 0 null values
Column Poverty has 79 null values
Column Unemployment has 155 null values


In [11]:
new_df = new_df.dropna()
new_df.head()
print(new_df.shape)

(7712, 16)


In [12]:
tract_df = new_df[["Census Tract", "Total Population", "ZIP", "Longitude", "Latitude"]]
tract_df.set_index('Census Tract')
tract_df.head()
# print(tract_df.shape)

,Census Tract,Total Population,ZIP,Longitude,Latitude
0,6019001100,3174,93706,-119.781696,36.709695
1,6071001600,6133,91761,-117.618013,34.057780
2,6019000200,3167,93706,-119.805504,36.735491
3,6077000801,6692,95203,-121.314524,37.940517
4,6019001500,2206,93725,-119.717843,36.681600


In [13]:
# tract_df.set_index('Census Tract')
# tract_df.head()

In [14]:
data_df = new_df.drop(['Total Population', 'ZIP', 'Longitude', 'Latitude'], axis=1)
data_df.head()

,Census Tract,Ozone,PM2.5,Diesel PM,Pesticides,Tox. Release,Traffic,Asthma,Low Birth Weight,Cardiovascular Disease,Poverty,Unemployment
0,6019001100,0.065,15.40,48.524,2.75,18551.957190,909.14,131.64,7.44,14.13,76.3,17.6
1,6071001600,0.062,13.31,38.556,1.37,7494.236622,782.26,60.66,7.04,12.94,72.5,12.3
2,6019000200,0.062,15.40,47.445,3.03,12454.948410,576.52,142.12,10.16,14.96,86.8,16.1
3,6077000801,0.046,12.54,24.117,12.93,2387.782922,1305.01,142.17,6.23,14.72,61.3,19.6
4,6019001500,0.065,15.40,18.846,3518.41,21790.706720,435.16,90.48,4.50,12.82,66.4,18.6


## Cleaning data for SQL

In [28]:
#Pollution DF seperated 
pollution_df = data_df[["Census Tract", "Ozone", "PM2.5", "Diesel PM", "Pesticides", "Tox. Release", "Traffic"]]

pollution_df.head()

,Census Tract,Ozone,PM2.5,Diesel PM,Pesticides,Tox. Release,Traffic
0,6019001100,0.065,15.40,48.524,2.75,18551.957190,909.14
1,6071001600,0.062,13.31,38.556,1.37,7494.236622,782.26
2,6019000200,0.062,15.40,47.445,3.03,12454.948410,576.52
3,6077000801,0.046,12.54,24.117,12.93,2387.782922,1305.01
4,6019001500,0.065,15.40,18.846,3518.41,21790.706720,435.16


In [29]:
# Set Index
pollution_df.set_index('Census Tract')

,Ozone,PM2.5,Diesel PM,Pesticides,Tox. Release,Traffic
Census Tract,,,,,,
6019001100,0.065,15.400000,48.524,2.75,18551.957190,909.14
6071001600,0.062,13.310000,38.556,1.37,7494.236622,782.26
6019000200,0.062,15.400000,47.445,3.03,12454.948410,576.52
6077000801,0.046,12.540000,24.117,12.93,2387.782922,1305.01
6019001500,0.065,15.400000,18.846,3518.41,21790.706720,435.16
...,...,...,...,...,...,...
6037262802,0.046,11.210000,11.286,0.16,809.425648,881.41
6053011602,0.035,5.344507,1.627,1.73,0.013115,441.77
6073008102,0.042,10.370000,3.455,0.00,103.305285,435.00


In [30]:
# Rename columns to fit into SQL
pollution_df.rename(columns = {'Census Tract' : 'Census_Tract', 'PM2.5': 'PM2_5', 'Diesel PM': 'Diesel_PM', 'Tox. Release' : 'Tox_Release'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [31]:
pollution_df.set_index('Census_Tract')

,Ozone,PM2_5,Diesel_PM,Pesticides,Tox_Release,Traffic
Census_Tract,,,,,,
6019001100,0.065,15.400000,48.524,2.75,18551.957190,909.14
6071001600,0.062,13.310000,38.556,1.37,7494.236622,782.26
6019000200,0.062,15.400000,47.445,3.03,12454.948410,576.52
6077000801,0.046,12.540000,24.117,12.93,2387.782922,1305.01
6019001500,0.065,15.400000,18.846,3518.41,21790.706720,435.16
...,...,...,...,...,...,...
6037262802,0.046,11.210000,11.286,0.16,809.425648,881.41
6053011602,0.035,5.344507,1.627,1.73,0.013115,441.77
6073008102,0.042,10.370000,3.455,0.00,103.305285,435.00


In [32]:
# New DF for socioeconomic data to load into SQL
socioeconomic_df = data_df[["Census Tract", "Asthma", "Low Birth Weight", "Cardiovascular Disease", "Poverty", "Unemployment", "Traffic"]]

In [33]:
# Socioeconimic fix column names for SQL
socioeconomic_df.rename(columns = {'Census Tract' : 'Census_Tract', 'Low Birth Weight': 'Low_Birth_Weight', 'Cardiovascular Disease': 'Cardiovascular_Disease'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [34]:
socioeconomic_df.set_index('Census_Tract')

,Asthma,Low_Birth_Weight,Cardiovascular_Disease,Poverty,Unemployment,Traffic
Census_Tract,,,,,,
6019001100,131.64,7.44,14.13,76.3,17.6,909.14
6071001600,60.66,7.04,12.94,72.5,12.3,782.26
6019000200,142.12,10.16,14.96,86.8,16.1,576.52
6077000801,142.17,6.23,14.72,61.3,19.6,1305.01
6019001500,90.48,4.50,12.82,66.4,18.6,435.16
...,...,...,...,...,...,...
6037262802,7.69,1.38,3.38,5.3,3.9,881.41
6053011602,11.65,1.02,2.63,14.7,4.5,441.77
6073008102,16.66,1.84,3.73,8.8,1.1,435.00


In [35]:
# Tract fix column names for SQL
tract_df.rename(columns = {'Census Tract' : 'Census_Tract', 'Total Population': 'Total_Population'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [36]:
tract_df.head()

,Census_Tract,Total_Population,ZIP,Longitude,Latitude
0,6019001100,3174,93706,-119.781696,36.709695
1,6071001600,6133,91761,-117.618013,34.057780
2,6019000200,3167,93706,-119.805504,36.735491
3,6077000801,6692,95203,-121.314524,37.940517
4,6019001500,2206,93725,-119.717843,36.681600


In [37]:
tract_df.set_index('Census_Tract')

,Total_Population,ZIP,Longitude,Latitude
Census_Tract,,,,
6019001100,3174,93706,-119.781696,36.709695
6071001600,6133,91761,-117.618013,34.057780
6019000200,3167,93706,-119.805504,36.735491
6077000801,6692,95203,-121.314524,37.940517
6019001500,2206,93725,-119.717843,36.681600
...,...,...,...,...
6037262802,3759,90272,-118.502456,34.045865
6053011602,5266,93923,-121.818271,36.540911
6073008102,3395,92037,-117.273466,32.819152


## Code to export to CSV

In [38]:
tract_df.to_csv('location_SQL.csv', index=False)
pollution_df.to_csv('pollution_SQL.csv', index=False)
socioeconomic_df.to_csv('socioeconomic_SQL.csv', index=False)

# Export to SQL DB

In [39]:


## Create Spark DBs
# spark_tract=spark.createDataFrame(tract_df) 
# spark_pollution=spark.createDataFrame(pollution_df) 
# spark_socio=spark.createDataFrame(socioeconomic_df) 

## Create Spark DBs from CSVs per Joseph Thompson (test)

# Load Location data as Spark DF
from pyspark import SparkFiles
url2 ="/content/location_SQL.csv"
spark.sparkContext.addFile(url2)
spark_location_df = spark.read.csv(SparkFiles.get("location_SQL.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
spark_location_df.show()


+------------+----------------+-----+-------------------+------------------+
|Census_Tract|Total_Population|  ZIP|          Longitude|          Latitude|
+------------+----------------+-----+-------------------+------------------+
|  6019001100|            3174|93706|-119.78169609999999|36.709695200000006|
|  6071001600|            6133|91761|       -117.6180131|        34.0577805|
|  6019000200|            3167|93706|       -119.8055044|        36.7354914|
|  6077000801|            6692|95203|       -121.3145235|        37.9405169|
|  6019001500|            2206|93725|-119.71784270000002|           36.6816|
|  6037204920|            2598|90023|-118.19749750000001|34.017500399999996|
|  6077000300|            2396|95203|-121.30207240000001|         37.952421|
|  6019001000|            4106|93706|-119.80431399999999| 36.69775070000001|
|  6037206050|            2146|90023|-118.22445309999999|34.029903600000004|
|  6019000400|            6343|93721|       -119.7762091|        36.7276563|

In [40]:
# Load pollution data into spark
url3 ="/content/pollution_SQL.csv"
spark.sparkContext.addFile(url3)
spark_pollution_df = spark.read.csv(SparkFiles.get("pollution_SQL.csv"), sep=",", header=True, inferSchema=True)


In [41]:
# Load Socioeconomic data into Spark
url4 ="/content/socioeconomic_SQL.csv"
spark.sparkContext.addFile(url4)
spark_socio_df = spark.read.csv(SparkFiles.get("socioeconomic_SQL.csv"), sep=",", header=True, inferSchema=True)

In [42]:
spark_socio_df.show(10)

+------------+------+----------------+----------------------+-------+------------+-------+
|Census_Tract|Asthma|Low_Birth_Weight|Cardiovascular_Disease|Poverty|Unemployment|Traffic|
+------------+------+----------------+----------------------+-------+------------+-------+
|  6019001100|131.64|            7.44|                 14.13|   76.3|        17.6| 909.14|
|  6071001600| 60.66|            7.04|                 12.94|   72.5|        12.3| 782.26|
|  6019000200|142.12|           10.16|                 14.96|   86.8|        16.1| 576.52|
|  6077000801|142.17|            6.23|                 14.72|   61.3|        19.6|1305.01|
|  6019001500| 90.48|             4.5|                 12.82|   66.4|        18.6| 435.16|
|  6037204920| 68.74|            7.35|                  10.4|   66.4|        11.6|2943.44|
|  6077000300|169.56|            8.36|                  12.7|   76.2|        14.4| 885.52|
|  6019001000|142.28|            7.83|                 14.96|   74.5|        20.0| 243.54|

In [43]:
spark_location_df.show(10)

+------------+----------------+-----+-------------------+------------------+
|Census_Tract|Total_Population|  ZIP|          Longitude|          Latitude|
+------------+----------------+-----+-------------------+------------------+
|  6019001100|            3174|93706|-119.78169609999999|36.709695200000006|
|  6071001600|            6133|91761|       -117.6180131|        34.0577805|
|  6019000200|            3167|93706|       -119.8055044|        36.7354914|
|  6077000801|            6692|95203|       -121.3145235|        37.9405169|
|  6019001500|            2206|93725|-119.71784270000002|           36.6816|
|  6037204920|            2598|90023|-118.19749750000001|34.017500399999996|
|  6077000300|            2396|95203|-121.30207240000001|         37.952421|
|  6019001000|            4106|93706|-119.80431399999999| 36.69775070000001|
|  6037206050|            2146|90023|-118.22445309999999|34.029903600000004|
|  6019000400|            6343|93721|       -119.7762091|        36.7276563|

In [44]:
spark_pollution_df.show(10)

+------------+-----+-----+------------------+----------+------------------+-------+
|Census_Tract|Ozone|PM2_5|         Diesel_PM|Pesticides|       Tox_Release|Traffic|
+------------+-----+-----+------------------+----------+------------------+-------+
|  6019001100|0.065| 15.4|48.523999999999994|      2.75|       18551.95719| 909.14|
|  6071001600|0.062|13.31|            38.556|      1.37| 7494.236621999999| 782.26|
|  6019000200|0.062| 15.4|            47.445|      3.03|12454.948409999999| 576.52|
|  6077000801|0.046|12.54|24.116999999999997|     12.93|       2387.782922|1305.01|
|  6019001500|0.065| 15.4|            18.846|   3518.41|       21790.70672| 435.16|
|  6037204920|0.046|12.89|             56.52|       0.0|       39040.17995|2943.44|
|  6077000300|0.046|13.44|             21.76|    172.49| 707.5361575000001| 885.52|
|  6019001000|0.065| 15.4|20.848000000000003|   1435.93| 6996.962409000001| 243.54|
|  6037206050|0.046|12.89|            53.958|       0.0|       10378.23648|2

In [45]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter your password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://datavizda.clscjqfhe802.us-east-2.rds.amazonaws.com:5432/AQP"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter your password··········


In [46]:

# Postgres driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-14 03:26:50--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-08-14 03:26:50 (6.62 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [47]:


# Write to AWS SQL DBs 
spark_location_df.write.jdbc(url=jdbc_url, table='Location', mode=mode, properties=config)

In [48]:
spark_pollution_df.write.jdbc(url=jdbc_url, table='Pollution_Exposure', mode=mode, properties=config)

In [49]:
spark_socio_df.write.jdbc(url=jdbc_url, table='Socioeconomic', mode=mode, properties=config)